In [ ]:
pip install datasets torch sentencepiece

In [ ]:
pip install -U transformers

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from transformers import MT5ForConditionalGeneration, MT5Tokenizer

# model_name = "lljllll2219/uk-mt5-base-xlsum-v3"
model_name = "SGaleshchuk/mT5-sum-news-ua"
tokenizer = MT5Tokenizer.from_pretrained(model_name)
model = MT5ForConditionalGeneration.from_pretrained(model_name)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [ ]:
google_drive_path = "/content/drive/MyDrive/data/telegram/"

# prepare dataset
from datasets import load_dataset
dataset_all = load_dataset("csv",
                       data_files=google_drive_path+"scrape.telegram_forwardsGT30_TokensGT100.csv")
dataset_all

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'chat', 'country', 'state', 'city', 'views', 'forwards', 'replies', 'messageText', 'predicted_class', 'messageDate'],
        num_rows: 1019
    })
})

In [ ]:
# sample_dataset = dataset_all['train'].select(range(20))
# sample_dataset = dataset_all['train'].shuffle(seed=42).select(range(20))
sample_dataset = dataset_all['train'].shuffle(seed=42).select(range(len(dataset_all['train'])))

In [ ]:
import re
def remove_emojis(text):
    emoji_pattern = re.compile(
        "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F700-\U0001F77F"  # alchemical symbols
        u"\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
        u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
        u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
        u"\U0001FA00-\U0001FA6F"  # Chess Symbols
        u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
        u"\U00002702-\U000027B0"  # Dingbats
        u"\U000024C2-\U0001F251"
        "]+", flags=re.UNICODE
    )
    return emoji_pattern.sub(r' ', text)

def remove_contact_info(text):
    # Remove email addresses
    text = re.sub(r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}', ' ', text)

    # Remove phone numbers
    phone_pattern = re.compile(
        r'(\+ 41|\+41|41)(\s?\d+)+',
        flags=re.UNICODE
    )
    text = phone_pattern.sub(' ', text)

    return text

def clean_text(example):
    example['cleanText'] = example['messageText']

    example['cleanText'] = example['cleanText'].lower()
    example['cleanText'] = re.sub(r'\(.*?\)', '', example['cleanText'])

    # example['cleanText'] = remove_emojis(example['cleanText']) # keep emoji?? research more TODO
    # example['cleanText'] = remove_contact_info(example['cleanText'])

    # # remove url
    example['cleanText'] = re.sub(r"\S*https?:\S*", " ", example['cleanText'])

    # must do this operation!!! Or will get different token ids (without\n vs with \\n) and greatly influence model performance!!
    example['cleanText'] = example['cleanText'].replace('\\n', ' ').replace('#',' ').replace('*', ' ')
    return example

sample_dataset = sample_dataset.map(clean_text)

In [ ]:
# check whether text is clean
print(sample_dataset['messageText'][0])
print(sample_dataset['cleanText'][0])

Прихисток у Швейцарії в листопаді 2022 року знайшла собі директорка Новокаховського професійного електротехнічного ліцею, колаборантка Карпенкова Олександра Василівна. Вона виявилася єдиною з керівників закладів профтехосвіти Нової Каховки, яка пішла на добровільну співпрацю з ворогом. \n\nЩе в липні 2022 року очолюваний нею ліцей розпочав підготовку до нового навчального року за «російськими програмами». Також вона зареєструвала свій заклад у російських реєстрах під назвою «государственное бюджетное образовательное учреждение среднего профессионального образования Херсонской области «Новокаховское техническое училище №7».\n\n«Після того як у жовтні окупанти розпочали так звану «евакуацію» з Нової Каховки, Олександра Карпенкова переїхала до Швейцарії. В цій країні, а також в Ізраїлі давно проживають її діти. Однак на цьому вона не зупинилась і звернулася до представників різних українських структур, аби ті допомогли їй виплутатися з історії про державну зраду. Директорка ліцею бідкалас

In [ ]:
def tokenize_function(examples):
    # shouldn't set these parameters: max_length = 516, padding="max_length", truncation=True
    return tokenizer(examples["cleanText"])

sample_dataset = sample_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/1019 [00:00<?, ? examples/s]

In [ ]:
import torch
import numpy as np
def get_predict(examples):
  output = model.generate(torch.from_numpy(np.array([examples['input_ids']])))
  examples['summary'] = tokenizer.decode(output[0], skip_special_tokens=True)
  return examples

sample_dataset = sample_dataset.map(get_predict)

Map:   0%|          | 0/1019 [00:00<?, ? examples/s]

In [ ]:
sample_dataset['summary'][:20]

['директорка новокаховського професійного електротехнічного ліцею, колядовиця карпенкова олександра василівна, яка стала єдиною з керівників закладів профтехосвіти нової каховки, яка пішла на добровільну співпрацю з ворогом, опинилася у швейцарії.',
 'у французькій школі école libre відкрили перші групи онлайн-уроків французької мови.',
 'швейцарія заборонила власникам українського посвідчення водія користуватися автомобілем зі швейцарською реєстрацією.',
 'фонд шкільних канікул у цюріху отримав пожертвування на літній табір для відпочинку для дітей з україни.',
 'європейський парламент підтримав розширення визнання українських посвідчень водія. тепер вони зможуть користуватись українськими посвідченнями водія без необхідності його обміну.',
 'у 13 магазинах швейцарії, які для цього відкрили спеціальний центр food corner, для домашніх тварин можна безкоштовно отримати корм.',
 'швейцарська асоціація добровольців (thmk) спільно з товариством українців в швейцарії 13 жовтня проведе в жен

In [ ]:
sample_dataset = sample_dataset.remove_columns(['input_ids', 'attention_mask'])
sample_dataset

Dataset({
    features: ['Unnamed: 0', 'chat', 'country', 'state', 'city', 'views', 'forwards', 'replies', 'messageText', 'predicted_class', 'messageDate', 'cleanText', 'summary'],
    num_rows: 1019
})

In [ ]:
output_path = "{}prediction_{}.csv".format(google_drive_path, model_name.split('/')[1])
sample_dataset.to_csv(output_path)

Creating CSV from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

6056709